
<table>
    <tr>
      <td>Grado en Ingeniería de Datos e Inteligencia Artificial - Facultad de Informática - UCM
      </td>
      <td>
      <img src="https://biblioteca.ucm.es/data/cont/media/www/pag-88746//escudo.jpg"  width=50/>
      </td>
     </tr>
</table>

## Adquisicion y preprocesamiento de datos - RafaC

### Carpetas y ficheros

### Índice
[Caminos](#Caminos)<br>
[Creación de carpetas](#Creación_de_carpetas)<br>
[Borrado](#Borrado)<br>
[Copia de ficheros](#Copia)<br>
[Ficheros .zip](#Zip)<br>

<a name="Caminos"></a>
#### Caminos

En cualquier sistema operativo los sistemas de almacenamiento permanentes (discusos, memorias flash, etc) van a tener un sistema de carpetas y ficheros:
    
    * Las carpetas pueden contener ficheros y otras carpetas
    * Los ficheros contienen datos
    
Debemos ser capaces de manejar estas estructuras en Python para automatizar nuestro proceso de adquisición de datos.

Primero veamos en qué carpeta nos encontramos ahora mismo (la del notebook de Python)

In [1]:
from pathlib import Path
path = Path.cwd()  
print(path)

e:\docencia\2223\APD\codigo\pandas


Añadimos un fichero al final del camino

In [2]:
import pandas as pd
df = pd.DataFrame({"nombre":["Bertoldo", "Herminia","Calixta","Aniceto"],"edad":[18,19,24,30]})
p = Path(path,"datos.csv")
df.to_csv(p,index=False)
print(p)

e:\docencia\2223\APD\codigo\pandas\datos.csv


¿El path existe? ¿es una carpeta o un fichero?

In [3]:
if p.exists():
    print(p,"existe",end=" ")
    if p.is_dir():
        print("y es una carpeta")
    else:
        if p.is_file():
            print("y es un fichero")
else:
    print(p,"no existe")

e:\docencia\2223\APD\codigo\pandas\datos.csv existe y es un fichero


Ya sabemos que p es un camino a un fichero, ahora vamos a ver algunas de sus características

In [4]:
p.name

'datos.csv'

In [5]:
p.stem

'datos'

In [6]:
p.suffix

'.csv'

Además podemos obtener [algunos datos](https://docs.python.org/3/library/os.html#os.stat_result) del fichero, por ejemplo su tamaño en bytes

In [7]:
estad = p.stat() # estadísticas varias
estad

os.stat_result(st_mode=33206, st_ino=2251799813687712, st_dev=3022412528, st_nlink=1, st_uid=0, st_gid=0, st_size=63, st_atime=1676923812, st_mtime=1676923812, st_ctime=1676888054)

In [8]:
estad.st_size 

63

Todas las carpetas y ficheros del path actual (que debe ser una carpeta)

In [40]:
def lista(path):
    print(path)
    espacio = " "*5
    for p in path.iterdir():
        if p.is_dir():
            print(espacio,p.name,": carpeta",sep="")
        else:
            if p.is_file():
                print(espacio,p.name,": fichero",sep="")
    print("="*40)
                
lista(path)                

e:\docencia\2223\APD\codigo\pandas
     .ipynb_checkpoints: carpeta
     1numpyShort.ipynb: fichero
     2series.ipynb: fichero
     3dataframes.ipynb: fichero
     4carga.ipynb: fichero
     5fechas.ipynb: fichero
     datos.csv: fichero
     paths.ipynb: fichero
     prueba.parquet: fichero
     raw: carpeta
     resumen.pdf: fichero
     series.ipynb: fichero


**Ejercicio** Vamos a crear un dataframe y a grabarlo en dos formatos

In [10]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
path = Path.cwd() 

# datos
numcolumnas = 100
filas = []
numfilas = 10000
for i in range(numfilas):
    filas.append(np.random.randint(1,5000,numcolumnas))

df = pd.DataFrame(filas, columns = ["c"+str(i) for i in range(numcolumnas)])
fichero_csv = Path(path,"prueba.csv")
fichero_parquet = Path(path,"prueba.parquet")
df.to_csv(fichero_csv,index=False)
df.to_parquet(fichero_parquet,index=False)

¿Cuál ocupa menos `fichero_csv`o `fichero_parquet`?

In [12]:
print(fichero_csv.stat().st_size)

4788976


In [13]:
print(fichero_parquet.stat().st_size)

3415548


Comparar el tiempo de lectura

In [14]:
%timeit pd.read_csv(fichero_csv)

74.9 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit pd.read_parquet(fichero_parquet)

9.68 ms ± 55.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<a name="Creación_de_carpetas"></a>
#### Creación de carpetas

Es un buen hábito tener los datos iniciarles en una carpeta `raw` y dejar los resultados en una carpeta `preprocess`. Para ello debemos ser capaces de crear carpetas si no existen

In [16]:
from pathlib import Path
path = Path.cwd()  
pathraw = Path(path,"raw")
if pathraw.exists():
    print("Ya existe")
else:
    print("No existe, lo creamos")
    pathraw.mkdir()
    if pathraw.exists():
        print("Creado")
    else:
        print("No se puede crear!!")

No existe, lo creamos
Creado


Si ya existe y no lo comprobamos  da error

In [17]:
#pathraw.mkdir() Si quitamos el comentario

Pero se puede hacer mejor con el atributo exists_ok, que indica que si ya existe no haga nada (y si no lo crea)

In [18]:
pathraw.mkdir(exist_ok=True)

Otro parámetro util para `mkdir` es parents=True, que creará además todos las carpetas intermedias. Esto es muy útil para crear una estructura compleja de ficheros

In [36]:
folders = ["raw/download/feb","raw/download/mar","raw/download/may"] 
for f in folders:
    newFolder = Path(path,f)
    newFolder.mkdir(exist_ok=True,parents=True)

<a name="Borrado"></a>
#### Borrado de carpetas y ficheros

En principio disponemos de dos métodos interesantes   
    
    * path.unlink() borra un fichero
    * path.rmdir() borra una carpeta

In [41]:
from pathlib import Path
path = Path.cwd() 
print("Antes")
lista(path)
fichero_csv = Path(path,"prueba.csv")
fichero_csv.unlink(missing_ok=True) # no da error aunque no exista
print("Después")
lista(path)

Antes
e:\docencia\2223\APD\codigo\pandas
     .ipynb_checkpoints: carpeta
     1numpyShort.ipynb: fichero
     2series.ipynb: fichero
     3dataframes.ipynb: fichero
     4carga.ipynb: fichero
     5fechas.ipynb: fichero
     datos.csv: fichero
     paths.ipynb: fichero
     prueba.parquet: fichero
     raw: carpeta
     resumen.pdf: fichero
     series.ipynb: fichero
Después
e:\docencia\2223\APD\codigo\pandas
     .ipynb_checkpoints: carpeta
     1numpyShort.ipynb: fichero
     2series.ipynb: fichero
     3dataframes.ipynb: fichero
     4carga.ipynb: fichero
     5fechas.ipynb: fichero
     datos.csv: fichero
     paths.ipynb: fichero
     prueba.parquet: fichero
     raw: carpeta
     resumen.pdf: fichero
     series.ipynb: fichero


En el caso de borrar una carpeta debe estar vacía:

In [21]:
from pathlib import Path
path = Path.cwd() 
raw_dir = Path(path,"raw")
#raw_dir.rmdir() si quitamos el comentario dará error

Si no queremos borrar elemento a elemento y queremos borrar todo un subárbol podemos usar la librería `shutil`

In [38]:
from shutil import rmtree
rmtree(raw_dir,ignore_errors=True) # para que no dé error si no existe

<a name="Copia"></a>
#### Copia de ficheros
Esta biblioteca también nos permite copiar ficheros de un destino a otro

Primero creamos algunas carpetas y un fichero de prueba 

In [42]:
folders = ["raw/download/feb","raw/download/mar","raw/download/may"] 
for f in folders:
    newFolder = Path(path,f)
    newFolder.mkdir(exist_ok=True,parents=True)
    
import pandas as pd
df = pd.DataFrame({"nombre":["Bertoldo", "Herminia","Calixta","Aniceto"],"edad":[18,19,24,30]})
p = Path(folders[0],"datos.csv")
df.to_csv(p,index=False)
lista(Path(folders[0]))  

raw\download\feb
     datos.csv: fichero


In [44]:
import pathlib
import shutil

print("Antes")
lista(Path(folders[1]))
origen=p
destino=Path(folders[1],"datoscopia.csv")
shutil.copy(origen, destino)
print("después")
lista(Path(folders[1]))


Antes
raw\download\mar
     datoscopia.csv: fichero
después
raw\download\mar
     datoscopia.csv: fichero


In [50]:
from shutil import rmtree
rmtree(raw_dir,ignore_errors=True) 

<a name="Zip"></a>
#### Ficheros .zip
Esta biblioteca también nos permite copiar ficheros de un destino a otro

Primero creamos algunas carpetas y un fichero de prueba 

In [53]:
from pathlib import Path
path = Path.cwd() 
download_dir = Path(path,"raw/download")
download_dir.mkdir(exist_ok=True,parents=True)

fichero = "meteo22.zip"
path_fich = Path(download_dir,fichero)

In [55]:
import requests
url = "https://github.com/RafaelCaballero/tdm/blob/master/datos/madrid/meteo22.zip?raw=true"
r = requests.get(url)
if r.status_code==200:
    with open(path_fich, 'wb') as f:
        f.write(r.content) # ahora lo grabamos localmente
    print("Grabado")
else:
    print("Error ",r.status_code)
    
lista(download_dir)    


Grabado
e:\docencia\2223\APD\codigo\pandas\raw\download
     meteo22.zip: fichero


Utilizamos `unpack_archive` que detecta el tipo de compresión y descomprime automáticamente. El primer parámetro es el fichero y el segundo la carpeta donde debe guardarse

In [56]:
import shutil
raw_dir = Path(path,"raw")
shutil.unpack_archive(path_fich, raw_dir)
lista(raw_dir)

e:\docencia\2223\APD\codigo\pandas\raw
     abr_meteo22.csv: fichero
     ago_meteo22.csv: fichero
     dic_meteo22.csv: fichero
     download: carpeta
     ene_meteo22.csv: fichero
     feb_meteo22.csv: fichero
     Interpretación_datos_meteorologicos.pdf: fichero
     jul_meteo22.csv: fichero
     jun_meteo22.csv: fichero
     mar_meteo22.csv: fichero
     may_meteo22.csv: fichero
     nov_meteo22.csv: fichero
     oct_meteo22.csv: fichero
     readme.txt: fichero
     sep_meteo22.csv: fichero
